In [8]:
# As usual, a bit of setup
import datetime
import itertools
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data
from cs231n.gradient_check import eval_numerical_gradient_array, eval_numerical_gradient
from cs231n.layers import *
from cs231n.fast_layers import *
from cs231n.solver import Solver
from cs231n.classifiers.convnet import *
from cs231n.cross_validation import cross_val_score

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [4]:
num_train = 5000
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

In [9]:
solver_config_temp = {
    "num_epochs": 10,
    "batch_size": 500,
    "update_rule": 'sgd_momentum',
    "optim_config": {
        'learning_rate': 1e-2,
    },
    "lr_decay": 0.9,
    "verbose": True,
    "print_every": 10
}


In [ ]:
use_batch_norm_arr = [False, True]
filter_num_arr = [16, 32]
hidden_layer_dim_arr = [100, 200]
reg_arr = [1e-2, 1e-3, 1e-4]
conv_layer_count_arr = [2, 3, 4]

print "Start at : ", datetime.datetime.now()
results = np.array((2*2*2*3*3 ,6), dtype=np.float32)
i = 0
for use_batch_norm, filter_num, hidden_layer_dim, reg,\
    conv_layer_count in itertools.product(use_batch_norm_arr, filter_num_arr, 
                                          hidden_layer_dim_arr, reg_arr, conv_layer_count_arr):
    conv_layer_config = ConvLayerConfig(num_filters=filter_num, use_batch_norm=use_batch_norm)
    hidden_layer_config = HiddenLayerConfig(hidden_layer_dim=hidden_layer_dim, use_batch_norm=use_batch_norm)

    # To add reg param here...
    conv_layer_configs = []
    for j in xrange(conv_layer_count):
        conv_layer_configs.append(conv_layer_config)
    model = MultipleLayerConvNet(conv_layer_configs, [hidden_layer_config, hidden_layer_config], 
                                 reg=reg, dtype=np.float32)
    score = cross_val_score(model, solver_config_temp, data['X_train'], data['y_train'], cv=5)
    results[i, :] = [use_batch_norm, filter_num, hidden_layer_dim, reg, conv_layer_count, score]
    print "cross validation score for [%d, %d, %d, %f, %d] is %f" %\
        (use_batch_norm, filter_num, hidden_layer_dim, reg, conv_layer_count, score)
    i+=1
    
print "End at : ", datetime.datetime.now()